<a href="https://colab.research.google.com/github/AmineBAA/Deep-Insights_Neural-Network/blob/main/Beta_VAE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # β - Variational Autoencoder

In [1]:
#IMPORTING
import numpy as np
import os
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm
import cv2

In [3]:
import keras
from keras.datasets import cifar100, cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

In [7]:
import keras

from keras import Sequential, Model,Input
from keras.layers import LeakyReLU
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import MaxPool2D, AvgPool2D
from keras.layers import UpSampling2D
from keras.layers import Lambda
from keras.layers import Activation
from keras.layers import BatchNormalization
from keras.layers import Flatten
from keras.layers import Reshape
from keras.layers import Add, Multiply
from keras.losses import mse, binary_crossentropy
import keras.backend as K

In [ ]:
###################################################################################################

In [92]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


In [93]:
# Hyperparameters
batch_size = 128
latent_dim = 10
beta = 4  # Adjust beta according to your preference
learning_rate = 1e-3
epochs = 50

In [94]:
# Data loading and preprocessing
transform = transforms.Compose([transforms.ToTensor()])
train_loader = DataLoader(datasets.MNIST('./data', train=True, download=True, transform=transform), batch_size=batch_size, shuffle=True)


100%|██████████| 9912422/9912422 [00:00<00:00, 74929687.59it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 23944592.57it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw




100%|██████████| 1648877/1648877 [00:00<00:00, 28301480.97it/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 13875111.99it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [95]:


# Define the VAE architecture
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.encoder = nn.Sequential(
            nn.Linear(28 * 28, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, latent_dim * 2)  # Two times latent_dim for mean and log-variance
        )

        self.decoder = nn.Sequential(
            nn.Linear(latent_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 256),
            nn.ReLU(),
            nn.Linear(256, 28 * 28),
            nn.Sigmoid()
        )

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def forward(self, x):
        # Encode
        h = self.encoder(x.view(-1, 28 * 28))
        mu, logvar = h[:, :latent_dim], h[:, latent_dim:]

        # Reparameterize
        z = self.reparameterize(mu, logvar)

        # Decode
        x_recon = self.decoder(z)
        return x_recon, mu, logvar

def loss_function(recon_x, x, mu, logvar):
    # Reconstruction loss (BCE) + KL divergence
    BCE = nn.BCELoss(reduction='sum')(recon_x, x.view(-1, 28 * 28))
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return BCE + beta * KLD








In [96]:
# Initialize the model and optimizer
model = VAE()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [97]:
# Training loop
model.train()
for epoch in range(epochs):
    total_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        total_loss += loss.item()
        optimizer.step()
    print(f'Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader.dataset):.4f}')

Epoch 1/50, Loss: 204.3957
Epoch 2/50, Loss: 171.3562
Epoch 3/50, Loss: 160.0588
Epoch 4/50, Loss: 155.5960
Epoch 5/50, Loss: 153.2197
Epoch 6/50, Loss: 151.6015
Epoch 7/50, Loss: 150.3514
Epoch 8/50, Loss: 149.3671
Epoch 9/50, Loss: 148.6537
Epoch 10/50, Loss: 147.9220
Epoch 11/50, Loss: 147.3724
Epoch 12/50, Loss: 146.9868
Epoch 13/50, Loss: 146.6168
Epoch 14/50, Loss: 146.2222
Epoch 15/50, Loss: 145.8516
Epoch 16/50, Loss: 145.5107
Epoch 17/50, Loss: 145.3101
Epoch 18/50, Loss: 145.0742
Epoch 19/50, Loss: 144.9618
Epoch 20/50, Loss: 144.6812
Epoch 21/50, Loss: 144.4317
Epoch 22/50, Loss: 144.3135
Epoch 23/50, Loss: 144.1433
Epoch 24/50, Loss: 144.0235
Epoch 25/50, Loss: 143.8246
Epoch 26/50, Loss: 143.7539
Epoch 27/50, Loss: 143.5891
Epoch 28/50, Loss: 143.5202
Epoch 29/50, Loss: 143.3951
Epoch 30/50, Loss: 143.3074
Epoch 31/50, Loss: 143.1838
Epoch 32/50, Loss: 143.0727
Epoch 33/50, Loss: 143.0381
Epoch 34/50, Loss: 143.0377
Epoch 35/50, Loss: 142.9420
Epoch 36/50, Loss: 142.7447
E

In [98]:
# Save the trained model
torch.save(model.state_dict(), 'beta_vae_model.pth')